#### Train with Convolution  - max pooling

In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
# load data created in -01
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscaleb

import numpy as np

def reformat(dataset, labels):
  # note here we reshape the input to 4D. 
  #the final dimension corresponding to the number of color channels
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

#### Prepare function to keep code cleaner

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# the strategy is to use strid of one and 2x2 padding with max pooling (same padding)
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [27]:
batch_size = 16
patch_size = 3
depth_1 = 32 # number of feature map
depth_2 = 64
num_hidden = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # First convolution layer 
  w_conv1 = weight_variable([patch_size, patch_size, num_channels, depth_1])
  b_conv1 = bias_variable([depth_1])
  
  # Second convolution layer 
  w_conv2 = weight_variable([patch_size, patch_size, depth_1, depth_2])
  b_conv2 = bias_variable([depth_2])
  
  # Fully connected layer
  w_fc1 = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth_2, num_hidden], stddev=0.1))
  # //4 because here we will have 2 max pool with strip of 2 same padding  
  b_fc1 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  w_out = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  b_out = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    h_conv1 = tf.nn.relu(conv2d(data, w_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    shape = h_pool2.get_shape().as_list()
    reshape = tf.reshape(h_pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, w_fc1) + b_fc1)
    return tf.matmul(hidden, w_out) + b_out
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [28]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  # tf.global_variables_initializer().run()
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.926723
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 25.018253
Minibatch accuracy: 12.5%
Validation accuracy: 10.1%
Minibatch loss at step 100: 3.157401
Minibatch accuracy: 12.5%
Validation accuracy: 36.8%
Minibatch loss at step 150: 0.947786
Minibatch accuracy: 75.0%
Validation accuracy: 69.7%
Minibatch loss at step 200: 0.648466
Minibatch accuracy: 75.0%
Validation accuracy: 72.5%
Minibatch loss at step 250: 1.391645
Minibatch accuracy: 68.8%
Validation accuracy: 74.2%
Minibatch loss at step 300: 1.022944
Minibatch accuracy: 68.8%
Validation accuracy: 75.3%
Minibatch loss at step 350: 0.954192
Minibatch accuracy: 75.0%
Validation accuracy: 75.2%
Minibatch loss at step 400: 0.546575
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 450: 1.542827
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.388128
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
M